In [1]:
from dotenv import load_dotenv
import sys

import faiss
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import DeepLake
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.llms import OpenAI
from langchain.chains import RetrievalQA

from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType
from langchain.vectorstores import FAISS

sys.path.append('..')
from backend.DB.db import Database

c:\Users\karen\Anaconda3\envs\hotel-reviews\lib\site-packages\deeplake\util\check_latest_version.py:32: UserWarning: A newer version of deeplake (3.6.7) is available. It's recommended that you update to the latest version using `pip install -U deeplake`.
  warnings.warn(


In [2]:
load_dotenv()
llm = OpenAI(model="text-davinci-003", temperature=0)
embeddings = OpenAIEmbeddings()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)


In [3]:
db = Database()
reviews = db.query_sql('SELECT title, review_text FROM raw_reviews')
texts = [f'{title}. {text}' for title, text in list(reviews)]
docs = text_splitter.create_documents(texts)
db = FAISS.from_documents(documents=docs, embedding=embeddings)


In [4]:
retrieval_qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type='stuff',
    retriever=db.as_retriever(search_type="similarity", search_kwargs={"k":2})
)

tools = [
    Tool(name='Retrival QA System',
         func=retrieval_qa.run,
         description="Answering questions")
]

agent = initialize_agent(tools,
                         llm,
                         agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
                         verbose=True)

In [5]:
response = agent.run("Tell me about reviews that talk about bugs?")
print(response)



> Entering new  chain...
 I need to find reviews that mention bugs.
Action: Retrival QA System
Action Input: Reviews that mention bugs
Observation:  It is possible that reviews online mentioned bugs, but it is not certain.
Thought: I need to find more specific information.
Action: Retrival QA System
Action Input: Reviews that mention bugs and their solutions
Observation:  I don't know.
Thought: I need to find reviews that discuss the solutions to the bugs.
Action: Retrival QA System
Action Input: Reviews that discuss solutions to bugs
Observation:  Reviews that discuss solutions to bugs might suggest pest control services or suggest that the place be reported to the health department.
Thought: I now know the final answer.
Final Answer: Reviews that discuss solutions to bugs might suggest pest control services or suggest that the place be reported to the health department.

> Finished chain.
Reviews that discuss solutions to bugs might suggest pest control services or suggest that the

In [6]:
response = agent.run("What do reviews say about breakfast?")
print(response)



> Entering new  chain...
 I should look for reviews about breakfast.
Action: Retrival QA System
Action Input: Reviews about breakfast
Observation:  The breakfast was limited but the food was good. The $14 voucher was nice but it didn't cover a meal and a coffee. It was still only a few bucks more which I can't complain about.
Thought: I now know the final answer.
Final Answer: Reviews say that the breakfast was limited but the food was good and the $14 voucher was nice but it didn't cover a meal and a coffee.

> Finished chain.
Reviews say that the breakfast was limited but the food was good and the $14 voucher was nice but it didn't cover a meal and a coffee.


In [7]:
response = agent.run("Can you summarize bad reviews?")
print(response)



> Entering new  chain...
 I need to find a way to summarize the reviews.
Action: Retrival QA System
Action Input: Summarize bad reviews
Observation:  The reviews mentioned that the pool doesn't open until 9am, which is inconvenient for early risers, and the bathroom sink was clogged. Additionally, the market was on vacation and the janitorial rating was taken down, and the blinds had to be lined up to block out the residents in the back of the room. The place was also a bit run-down for the price.
Thought: I now know the final answer
Final Answer: The reviews mentioned that the pool doesn't open until 9am, which is inconvenient for early risers, the bathroom sink was clogged, the market was on vacation, the janitorial rating was taken down, the blinds had to be lined up to block out the residents in the back of the room, and the place was a bit run-down for the price.

> Finished chain.
The reviews mentioned that the pool doesn't open until 9am, which is inconvenient for early risers

In [8]:
response = agent.run("What is the average sentiment about the cleanliness of rooms?")
print(response)



> Entering new  chain...
 I need to find out what people are saying about the cleanliness of rooms.
Action: Retrival QA System
Action Input: Reviews about the cleanliness of rooms
Observation:  Reviews about the cleanliness of rooms were generally positive, with people noting that the rooms were clean and everything worked. Janitorial took the rating down, but people were still satisfied with the cleanliness of the rooms.
Thought: I need to calculate the average sentiment of the reviews.
Action: Retrival QA System
Action Input: Average sentiment of reviews about the cleanliness of rooms
Observation:  Positive
Thought: I now know the final answer.
Final Answer: The average sentiment about the cleanliness of rooms is positive.

> Finished chain.
The average sentiment about the cleanliness of rooms is positive.


In [10]:
response = agent.run("Were there any complaints were there about the cleanliness of rooms?")
print(response)



> Entering new  chain...
 I should look for reviews or feedback from customers.
Action: Retrival QA System
Action Input: Reviews of hotel rooms
Observation:  Reviews of the hotel rooms were generally positive. People were pleased with the room size and cleanliness, and the hotel provided a white noise machine to address any road noise concerns. There was a mini fridge in the room, and the hotel had a good vibe and was family friendly.
Thought: I now know the final answer
Final Answer: Most reviews of the hotel rooms were positive, with customers praising the room size and cleanliness.

> Finished chain.
Most reviews of the hotel rooms were positive, with customers praising the room size and cleanliness.


In [9]:
response = agent.run("What aspects of stays did receive complaints?")
print(response)



> Entering new  chain...
 I need to find out what people have complained about when it comes to stays.
Action: Retrival QA System
Action Input: What complaints have people had about stays?
Observation:  People have complained about not having their patio swept, being placed next to an adjoining room with a pet and loud guests, not having enough consideration for guests without pets, and not having good window coverings.
Thought: I now know the final answer.
Final Answer: People have complained about not having their patio swept, being placed next to an adjoining room with a pet and loud guests, not having enough consideration for guests without pets, and not having good window coverings.

> Finished chain.
People have complained about not having their patio swept, being placed next to an adjoining room with a pet and loud guests, not having enough consideration for guests without pets, and not having good window coverings.


In [12]:
response = agent.run("Can you provide an overall summary of the reviews?")
print(response)



> Entering new  chain...
 I need to find a way to summarize the reviews.
Action: Retrival QA System
Action Input: Summarize reviews
Observation:  Reviews of the room were generally positive, noting that the room was clean and had all the necessary amenities. However, some reviews noted that the window view was disappointing and that housekeeping only operated on alternate days.
Thought: I now know the final answer
Final Answer: The reviews of the room were generally positive, noting that the room was clean and had all the necessary amenities, but some reviews noted that the window view was disappointing and that housekeeping only operated on alternate days.

> Finished chain.
The reviews of the room were generally positive, noting that the room was clean and had all the necessary amenities, but some reviews noted that the window view was disappointing and that housekeeping only operated on alternate days.
